In [3]:
import pandas as pd

data = pd.read_csv('../laptops_train.csv')
display(data)

,Manufacturer,Model Name,Category,Screen Size,Screen,CPU,RAM,Storage,GPU,Operating System,Operating System Version,Weight,Price
0,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37kg,11912523.48
1,Apple,Macbook Air,Ultrabook,"13.3""",1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34kg,7993374.48
2,HP,250 G6,Notebook,"15.6""",Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86kg,5112900.00
3,Apple,MacBook Pro,Ultrabook,"15.4""",IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83kg,22563005.40
4,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37kg,16037611.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,Dell,Alienware 17,Gaming,"17.3""",Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,32GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows,10,4.42kg,24897600.00
973,Toshiba,Tecra A40-C-1DF,Notebook,"14.0""",Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows,10,1.95kg,10492560.00
974,Asus,Rog Strix,Gaming,"17.3""",Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows,10,2.73kg,18227710.80
975,HP,Probook 450,Notebook,"15.6""",IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.70GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce 930MX,Windows,10,2.04kg,8705268.00


In [4]:
category_counts = data['Category'].value_counts()
tam_cat = len(category_counts)

In [5]:
probabilities = category_counts / len(data)

for p in probabilities:
    print(p)

0.5619242579324463
0.1555783009211873
0.1535312180143296
0.08700102354145343
0.02456499488229273
0.017400204708290685


In [8]:
from math import log2

def entropy(size_categ):
    sumc = 0
    for p in probabilities:
        sumc += -p * log2(p)
    return sumc

def maxEntropy(size_categ):
    maxResult = log2(size_categ)
    return maxResult

print("The entropy is:", entropy(len(category_counts)))
print("The maximum entropy is:", maxEntropy(len(category_counts)))

The entropy is: 1.8394863920687647
The maximum entropy is: 2.584962500721156


In [9]:
def allEntropy():
    for colName in data.columns:
        # Every class in a column
        classes = data[colName].value_counts()
        
        # Probability of each column
        probabilities = classes / len(data)
        
        # Entropy of each column
        eachEntropy = 0
        for p in probabilities:
            eachEntropy += -p * log2(p)
            maxEachEntropy = log2(len(probabilities))
            
        print(f"Entropy of the column {colName}: {eachEntropy}. Maximum: {maxEachEntropy}")
        
        # Entropy of all data
        entropy = 0
        maxAllEntropy = 0
        
        entropy += eachEntropy
        maxAllEntropy += maxEachEntropy
        
    print(f"\nEntropy of all data: {entropy}. Maximum: {maxAllEntropy}")

allEntropy()

Entropy of the column Manufacturer: 2.9185810340878815. Maximum: 4.247927513443585
Entropy of the column Model Name: 8.306483404105897. Maximum: 8.930737337562887
Entropy of the column Category: 1.8394863920687647. Maximum: 2.584962500721156
Entropy of the column Screen Size: 2.218326616944687. Maximum: 4.169925001442312
Entropy of the column Screen: 2.8651300327722895. Maximum: 5.247927513443585
Entropy of the column CPU: 4.754148641969021. Maximum: 6.727920454563199
Entropy of the column RAM: 1.8702493741995636. Maximum: 3.0
Entropy of the column  Storage: 3.267737600396686. Maximum: 5.169925001442312
Entropy of the column GPU: 4.82964674294513. Maximum: 6.614709844115208
Entropy of the column Operating System: 0.8787680493804171. Maximum: 2.807354922057604
Entropy of the column Operating System Version: 0.3702363755911487. Maximum: 2.0
Entropy of the column Weight: 6.417250720932384. Maximum: 7.3750394313469245
Entropy of the column Price: 8.988454655569397. Maximum: 9.3196721209469

In [13]:
from sklearn.neighbors import KNeighborsClassifier

test = pd.read_csv('../laptops_test.csv')

# Discretiza dados quantitativos
def discretizeColumn(column, testColumn):
    knn = KNeighborsClassifier(n_neighbors=10)
    columnValues = column.values.reshape(-1, 1)
    labels = pd.qcut(columnValues.flatten(), q=10, labels=False, duplicates='drop')
    knn.fit(columnValues, labels)
    testColumnValues = testColumn.values.reshape(-1, 1)
    discretizedValues = knn.predict(testColumnValues)
    return discretizedValues


# Calcula a entropia de cada coluna
def allEntropy():
    for colName in data.columns:
        column = data[colName]
        testColumn = test[colName]
        if column.dtype == 'object':
            # Categorical column
            classes = column.value_counts()
            probabilities = classes / len(data)
            eachEntropy = 0
            for p in probabilities:
                eachEntropy += -p * log2(p)
            maxEachEntropy = log2(len(probabilities))
        else:
            # Quantitative column
            discColumn = discretizeColumn(column,testColumn)
            classes = pd.Series(discColumn).value_counts()
            probabilities = classes / len(test)
            eachEntropy = 0
            for p in probabilities:
                eachEntropy += -p * log2(p)
            maxEachEntropy = log2(len(probabilities))

        print(f"Entropy of the column {colName}: {eachEntropy}. Maximum: {maxEachEntropy}")

allEntropy()

Entropy of the column Manufacturer: 2.9185810340878815. Maximum: 4.247927513443585
Entropy of the column Model Name: 8.306483404105897. Maximum: 8.930737337562887
Entropy of the column Category: 1.8394863920687647. Maximum: 2.584962500721156
Entropy of the column Screen Size: 2.218326616944687. Maximum: 4.169925001442312
Entropy of the column Screen: 2.8651300327722895. Maximum: 5.247927513443585
Entropy of the column CPU: 4.754148641969021. Maximum: 6.727920454563199
Entropy of the column RAM: 1.8702493741995636. Maximum: 3.0
Entropy of the column  Storage: 3.267737600396686. Maximum: 5.169925001442312
Entropy of the column GPU: 4.82964674294513. Maximum: 6.614709844115208
Entropy of the column Operating System: 0.8787680493804171. Maximum: 2.807354922057604
Entropy of the column Operating System Version: 0.3702363755911487. Maximum: 2.0
Entropy of the column Weight: 6.417250720932384. Maximum: 7.3750394313469245
Entropy of the column Price: 3.2806381645372467. Maximum: 3.321928094887